## SQL-запросы для анализа данных агенства недвижимости по продажам квартир в Санкт-Петербурге и ЛО (реализовано в DBeaver).

### Задача 1. Время активности объявлений

In [ ]:
-- Определим аномальные значения для таблицы flats:
WITH limits AS (
    SELECT  
        PERCENTILE_DISC(0.99) WITHIN GROUP (ORDER BY total_area) AS total_area_limit,
        PERCENTILE_DISC(0.99) WITHIN GROUP (ORDER BY rooms) AS rooms_limit,
        PERCENTILE_DISC(0.99) WITHIN GROUP (ORDER BY balcony) AS balcony_limit,
        PERCENTILE_DISC(0.99) WITHIN GROUP (ORDER BY ceiling_height) AS ceiling_height_limit_h,
        PERCENTILE_DISC(0.01) WITHIN GROUP (ORDER BY ceiling_height) AS ceiling_height_limit_l
    FROM real_estate.flats     
),
-- Найдём id объявлений, которые не содержат выбросы:
filtered_id AS(
    SELECT id
    FROM real_estate.flats  
    WHERE 
        total_area < (SELECT total_area_limit FROM limits)
        AND (rooms < (SELECT rooms_limit FROM limits) OR rooms IS NULL)
        AND (balcony < (SELECT balcony_limit FROM limits) OR balcony IS NULL)
        AND ((ceiling_height < (SELECT ceiling_height_limit_h FROM limits)
            AND ceiling_height > (SELECT ceiling_height_limit_l FROM limits)) OR ceiling_height IS NULL)
    ),
-- объединим данные из остальных таблиц и добавим доп.параметры:
sheet_1 AS (
   SELECT f.id,
          c.city,
         CASE WHEN city = 'Санкт-Петербург' THEN 'СПб' ELSE 'ЛО' END AS region_city,
         a.last_price/f.total_area AS price_per_meter,
         CASE WHEN days_exposition <= 30 THEN 'до 30 дней' 
           WHEN days_exposition <= 90 THEN 'до 90 дней'
           WHEN days_exposition <= 180 THEN 'до 180 дней' 
           WHEN days_exposition IS NULL THEN 'нет данных' ELSE '180 дней+'
           END AS selling_period,
           f.total_area,
           f.living_area,
           f.kitchen_area,
           f.rooms,
           f.balcony,
           f.floors_total,
           f.ceiling_height,
           f.is_apartment,
           f.open_plan,
           f.airports_nearest,
           f.parks_around3000,
           f.ponds_around3000,
           EXTRACT (YEAR FROM first_day_exposition) AS year_exposition
   FROM real_estate.flats AS f
   LEFT JOIN real_estate.advertisement AS a USING (id)
   LEFT JOIN real_estate.city AS c USING (city_id)
   LEFT JOIN real_estate.type AS t USING (type_id)
   WHERE id IN (SELECT * FROM filtered_id) AND t.type = 'город'),
--соберем сводную таблицу с характеристиками
sheet_2 AS(
   SELECT CASE WHEN selling_period = 'до 30 дней'THEN 1
            WHEN selling_period = 'до 90 дней' THEN 2
            WHEN selling_period = 'до 180 дней' THEN 3 
            WHEN selling_period = '180 дней+' THEN 4 ELSE 5 END AS №,
       region_city AS region,
       selling_period AS days_exposition,
       COUNT (id) AS listings_count,
       ROUND (COUNT(id) * 100.0 / SUM(COUNT(id)) OVER (PARTITION BY region_city),2) AS "share_by_region, %",
       AVG (price_per_meter)::integer AS avg_price_per_meter,
       ROUND (AVG (total_area::NUMERIC),1) AS avg_total_area,
       ROUND (AVG (living_area::NUMERIC),1) AS avg_living_area,
       ROUND (AVG (kitchen_area::NUMERIC),1) AS avg_kitchen_area,
       PERCENTILE_DISC (0.5) WITHIN GROUP (ORDER BY rooms) AS rooms_median,
       PERCENTILE_DISC (0.5) WITHIN GROUP (ORDER BY balcony) AS balcony_median,
       PERCENTILE_DISC (0.5) WITHIN GROUP (ORDER BY floors_total) AS floors_total_median,
       ROUND (AVG (ceiling_height::NUMERIC),2) AS avg_ceiling_height,
       PERCENTILE_DISC (0.5) WITHIN GROUP (ORDER BY parks_around3000) AS parks_median,
       PERCENTILE_DISC (0.5) WITHIN GROUP (ORDER BY ponds_around3000) AS ponds_median,
       AVG (airports_nearest)::integer AS avg_airport_nearest,
       SUM (open_plan) AS open_plan,
       SUM (is_apartment) AS is_apartment
   FROM sheet_1
   WHERE year_exposition BETWEEN 2015 AND 2018
   GROUP BY region_city, selling_period
   ORDER BY region_city DESC, №)
SELECT *,
        ROUND (open_plan/listings_count::NUMERIC,3) AS share_open_plan,
        ROUND (is_apartment/listings_count::NUMERIC,3) AS share_apartment
FROM sheet_2;

--Oтдельно посмотрим, в каких годах размещены объявления, которые не имеют срока размещения:
-- Определим аномальные значения для таблицы flats:
WITH limits AS (
    SELECT  
        PERCENTILE_DISC(0.99) WITHIN GROUP (ORDER BY total_area) AS total_area_limit,
        PERCENTILE_DISC(0.99) WITHIN GROUP (ORDER BY rooms) AS rooms_limit,
        PERCENTILE_DISC(0.99) WITHIN GROUP (ORDER BY balcony) AS balcony_limit,
        PERCENTILE_DISC(0.99) WITHIN GROUP (ORDER BY ceiling_height) AS ceiling_height_limit_h,
        PERCENTILE_DISC(0.01) WITHIN GROUP (ORDER BY ceiling_height) AS ceiling_height_limit_l
    FROM real_estate.flats     
),
-- Найдём id объявлений, которые не содержат выбросы:
filtered_id AS(
    SELECT id
    FROM real_estate.flats  
    WHERE 
        total_area < (SELECT total_area_limit FROM limits)
        AND (rooms < (SELECT rooms_limit FROM limits) OR rooms IS NULL)
        AND (balcony < (SELECT balcony_limit FROM limits) OR balcony IS NULL)
        AND ((ceiling_height < (SELECT ceiling_height_limit_h FROM limits)
            AND ceiling_height > (SELECT ceiling_height_limit_l FROM limits)) OR ceiling_height IS NULL)
    ),
-- объединим данные из остальных таблиц и добавим доп.параметры:
sheet_1 AS (
   SELECT f.id,
          c.city,
         CASE WHEN city = 'Санкт-Петербург' THEN 'СПб' ELSE 'ЛО' END AS region_city,
         a.last_price/f.total_area AS price_per_meter,
         CASE WHEN days_exposition <= 30 THEN 'до 30 дней' 
           WHEN days_exposition <= 90 THEN 'до 90 дней'
           WHEN days_exposition <= 180 THEN 'до 180 дней' 
           WHEN days_exposition IS NULL THEN 'нет данных' ELSE '180 дней+'
           END AS selling_period,
           f.total_area,
           f.living_area,
           f.kitchen_area,
           f.rooms,
           f.balcony,
           f.floors_total,
           f.ceiling_height,
           f.is_apartment,
           f.open_plan,
           f.airports_nearest,
           f.parks_around3000,
           f.ponds_around3000,
           EXTRACT (YEAR FROM first_day_exposition) AS year_exposition
   FROM real_estate.flats AS f
   LEFT JOIN real_estate.advertisement AS a USING (id)
   LEFT JOIN real_estate.city AS c USING (city_id)
   LEFT JOIN real_estate.type AS t USING (type_id)
   WHERE id IN (SELECT * FROM filtered_id) AND t.type = 'город')

   
--отдельно посмотрим, в каких годах размещены объявления, которые не имеют срока размещения:
SELECT region_city,
       year_exposition,
       COUNT (*) AS listings_count,
       ROUND (COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (PARTITION BY region_city),1) AS "share_by_region, %"
FROM sheet_1
WHERE selling_period = 'нет данных' AND year_exposition BETWEEN 2015 AND 2018
GROUP BY region_city, year_exposition
ORDER BY region_city DESC, year_exposition;

### Задача 2. Сезонность объявлений. 

In [ ]:
-- для анализа использованы только полные года (2015-2018) 

-- Определим аномальные значения для таблицы flats:
WITH limits AS (
    SELECT  
        PERCENTILE_DISC(0.99) WITHIN GROUP (ORDER BY total_area) AS total_area_limit,
        PERCENTILE_DISC(0.99) WITHIN GROUP (ORDER BY rooms) AS rooms_limit,
        PERCENTILE_DISC(0.99) WITHIN GROUP (ORDER BY balcony) AS balcony_limit,
        PERCENTILE_DISC(0.99) WITHIN GROUP (ORDER BY ceiling_height) AS ceiling_height_limit_h,
        PERCENTILE_DISC(0.01) WITHIN GROUP (ORDER BY ceiling_height) AS ceiling_height_limit_l
    FROM real_estate.flats     
),
-- Найдём id объявлений, которые не содержат выбросы:
filtered_id AS(
    SELECT id
    FROM real_estate.flats  
    WHERE 
        total_area < (SELECT total_area_limit FROM limits)
        AND (rooms < (SELECT rooms_limit FROM limits) OR rooms IS NULL)
        AND (balcony < (SELECT balcony_limit FROM limits) OR balcony IS NULL)
        AND ((ceiling_height < (SELECT ceiling_height_limit_h FROM limits)
            AND ceiling_height > (SELECT ceiling_height_limit_l FROM limits)) OR ceiling_height IS NULL)
    ),
-- добавим нужные данные из остальных таблиц:
sheet_1 AS (
   SELECT f.id,
         a.last_price/f.total_area AS price_per_meter,
         f.total_area,
         EXTRACT (YEAR FROM first_day_exposition) AS year_exposition,
         a.first_day_exposition,
         a.first_day_exposition + INTERVAL '1 day' * a.days_exposition AS last_day_exposition,
         a.days_exposition
   FROM real_estate.flats AS f
   LEFT JOIN real_estate.advertisement AS a USING (id)
   WHERE id IN (SELECT * FROM filtered_id)),
--посчитаем данные для опубликованных объявлений:
sheet_2 AS (
   SELECT TO_CHAR(first_day_exposition,'month') AS month,
          COUNT (*) AS published_count,
          RANK() OVER (ORDER BY COUNT(*) DESC) AS rank_published,
          AVG(price_per_meter)::integer AS avg_price_per_meter_published,
          RANK() OVER (ORDER BY AVG(price_per_meter) DESC) AS rank_price_published,
          ROUND(AVG (total_area::NUMERIC),2) AS avg_total_area_published     
   FROM sheet_1
   WHERE year_exposition > 2014 AND  year_exposition < 2019
   GROUP BY TO_CHAR(first_day_exposition,'month')),
-- посчитаем данные для снятых объявлений:
sheet_3 AS (
   SELECT TO_CHAR(last_day_exposition,'month') AS month,
          COUNT (*) AS removed_count,
          RANK() OVER (ORDER BY COUNT(*) DESC) AS rank_removed,
          AVG(price_per_meter)::integer AS avg_price_per_meter_removed,
          RANK() OVER (ORDER BY AVG(price_per_meter) DESC) AS rank_price_removed,
          ROUND(AVG (total_area::NUMERIC),2) AS avg_total_area_removed     
   FROM sheet_1
   WHERE year_exposition > 2014 AND year_exposition < 2019 AND days_exposition IS NOT NULL
   GROUP BY TO_CHAR(last_day_exposition,'month'))
--посчитаем особенности сезонности
SELECT s2.month,
       published_count,
       rank_published,
       removed_count,
       rank_removed,
       avg_price_per_meter_published,
       rank_price_published,
       avg_price_per_meter_removed,
       rank_price_removed,
       avg_total_area_published,
       avg_total_area_removed
FROM sheet_2 AS s2
JOIN sheet_3 USING (month)
ORDER BY rank_published;

### Задача 3. Анализ рынка недвижимости Ленобласти - топ-15 населенных пунктов по количеству размещенных объявлений.

In [ ]:
-- Определим аномальные значения для таблицы flats:
WITH limits AS (
    SELECT  
        PERCENTILE_DISC(0.99) WITHIN GROUP (ORDER BY total_area) AS total_area_limit,
        PERCENTILE_DISC(0.99) WITHIN GROUP (ORDER BY rooms) AS rooms_limit,
        PERCENTILE_DISC(0.99) WITHIN GROUP (ORDER BY balcony) AS balcony_limit,
        PERCENTILE_DISC(0.99) WITHIN GROUP (ORDER BY ceiling_height) AS ceiling_height_limit_h,
        PERCENTILE_DISC(0.01) WITHIN GROUP (ORDER BY ceiling_height) AS ceiling_height_limit_l
    FROM real_estate.flats     
),
-- Найдём id объявлений, которые не содержат выбросы:
filtered_id AS(
    SELECT id
    FROM real_estate.flats  
    WHERE 
        total_area < (SELECT total_area_limit FROM limits)
        AND (rooms < (SELECT rooms_limit FROM limits) OR rooms IS NULL)
        AND (balcony < (SELECT balcony_limit FROM limits) OR balcony IS NULL)
        AND ((ceiling_height < (SELECT ceiling_height_limit_h FROM limits)
            AND ceiling_height > (SELECT ceiling_height_limit_l FROM limits)) OR ceiling_height IS NULL)),
-- добавим данные из остальных таблиц и разделим на категории для анализа, разделим их на ЛО и СПб и выделим год из даты публикации объявления:
sheet_1 AS (
   SELECT c.city,
          f.total_area,
          a.last_price/f.total_area AS price_per_meter,
          a.days_exposition,
          EXTRACT (YEAR FROM a.first_day_exposition) AS year_exposition
   FROM real_estate.flats AS f
   LEFT JOIN real_estate.advertisement AS a USING (id)
   LEFT JOIN real_estate.city AS c USING (city_id)
   WHERE id IN (SELECT * FROM filtered_id) AND city <> 'Санкт-Петербург')
--посчитаем статистику по городам ЛО
SELECT city,
       COUNT (*) AS listings_count,
       ROUND(COUNT (*) FILTER (WHERE days_exposition IS NOT NULL)/COUNT (*)::numeric * 100.0, 2) AS share_removed,
       AVG(price_per_meter)::integer AS avg_price_per_meter,
       ROUND(AVG(total_area::NUMERIC), 1) AS avg_total_area,
       AVG(days_exposition)::integer AS avg_days_exposition
FROM sheet_1
WHERE year_exposition BETWEEN 2015 AND 2018
GROUP BY city
ORDER BY listings_count DESC
LIMIT 15;